In [1]:
import os
import cv2
import sys
import glob
import pandas as pd
import numpy as np

In [11]:
def splitImagesByAspectRatio(path):
    for folder in os.listdir(path):
        for file in os.listdir(path + folder):
            img = cv2.imread(path + folder + "/" + file)
            height, width, channels = img.shape
            aspect_ratio = width/height
            createSquaredImages(img, aspect_ratio, path,folder,file)

def createSquaredImages(img, aspect_ratio, path,folder,file):
    if(aspect_ratio == 4/3):
        square_shape = img.shape[0] // 3
    else:
        square_shape = img.shape[0] // 9
    count = 0
    if not os.path.exists(os.path.join(path,folder)):
        os.mkdir(os.path.join(path,folder))
    for i in np.arange(0,img.shape[0],square_shape):
        for j in np.arange(0, img.shape[1],square_shape):
            subfig = img[i:i+square_shape,j:j+square_shape]
            if j >= img.shape[1]/2:
                subfig = cv2.flip(subfig,1)
            figname = os.path.join(path, folder) + "/" + file.split('.')[0] + '-' + str(count) + '.' + file.split('.')[1]
            print(figname)
            cv2.imwrite(figname, subfig)
            count += 1               

In [3]:
path = '/media/newdrive/'
new_folder = os.path.join(path, 'processed_sss')
print(new_folder)
if not os.path.exists(new_folder):
    os.mkdir(new_folder)
count = 0
images = []
for img in glob.iglob(path + '**/*.png', recursive=True):
    count += 1
    print(count, img)
    images.append(img)

/media/newdrive/processed_sss
1 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-4.png
2 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-10.png
3 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-15.png
4 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-19.png
5 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-18.png
6 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-9.png
7 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-8.png
8 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-20.png
9 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-16.png
10 /media/newdrive/lauv-noptilus-3/20

In [4]:
folders = []
dfs = []
for img in images:
    f = '/'.join(img.split('/')[:-3])
    if(f not in folders):
        if os.path.exists(os.path.join(f,'mra/filtered_sss_images/positions.csv')):
            print(f)
            print(os.path.join(f,'mra/filtered_sss_images/positions.csv'))
            dfs.append(os.path.join(f,'mra/filtered_sss_images/positions.csv'))
            folders.append(f)


/media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam
/media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180711/115633_np3_cam2
/media/newdrive/lauv-noptilus-3/20180711/115633_np3_cam2/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180710/100154_A003_NP3
/media/newdrive/lauv-noptilus-3/20180710/100154_A003_NP3/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180710/133936_cam_np3_2
/media/newdrive/lauv-noptilus-3/20180710/133936_cam_np3_2/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180710/122322_cam_np3
/media/newdrive/lauv-noptilus-3/20180710/122322_cam_np3/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180501/113610_cam_survey_1
/media/newdrive/lauv-noptilus-3/20180501/113610_cam_survey_1/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180501/104728_cam-np3
/media/newdrive/lauv-nopti

In [5]:
len(dfs)

24

In [ ]:
df = pd.DataFrame(columns=['filename','timestamp','latitude','longitude','distance','depth','altitude','roll','pitch',
                           'auv','date'])

i = 0
for d in dfs:
    ps = d.split('/')
    auv = ps[3]
    date = ps[4][-2:] + '/' + ps[4][-4:-2] + '/' + ps[4][:4]
    log_df = pd.read_csv(d)
    for index, row in log_df.iterrows():
        df.loc[i] = [row['filename'] + '-' + str(i%12), row['timestamp'], row['latitude'], row['longitude'], 
                   row['distance'], row['depth'], row['altitude'], row['roll'], row['pitch'], auv, date]
        i += 1

In [ ]:
print(i , len(df))


In [17]:
#df.to_csv(os.path.join(new_folder, 'df.csv'), index=False)
df = pd.read_csv(os.path.join(new_folder, 'df.csv'))

In [12]:
for imgname in images:
    img = cv2.imread(imgname)
    ap = 4/3
    folder = imgname.split('/')[5]
    createSquaredImages(img, ap, new_folder, folder, imgname.split('/')[-1])
    

/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-0.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-1.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-2.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-3.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-4.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-5.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-6.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-7.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-8.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-9.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-10.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-11.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-10-0.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-10-1.png
/media/newdrive/processed_sss/105036_np3_cam

/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-9.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-10.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-11.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-0.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-1.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-2.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-3.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-4.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-5.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-6.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-7.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-8.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-9.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-6-10.png
/media/newdrive/processed_sss/105036_np3_cam/

/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-10-7.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-10-8.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-10-9.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-10-10.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-10-11.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-0.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-1.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-2.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-3.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-4.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-5.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-6.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-7.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-3-8.png
/media/newdri

/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-21-6.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-21-7.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-21-8.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-21-9.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-21-10.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-21-11.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-6-0.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-6-1.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-6-2.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-6-3.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-6-4.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-6-5.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-6-6.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-6-7.png
/media/newdr

/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-30-6.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-30-7.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-30-8.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-30-9.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-30-10.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-30-11.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-19-0.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-19-1.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-19-2.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-19-3.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-19-4.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-19-5.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-19-6.png
/media/newdrive/processed_sss/115633_np3_cam2/115633_np3_cam2-19-7.png
/med

...

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-76-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-17-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-17-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-17-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-17-3.png
/med

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-14-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-20-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-20-1.png
/med

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-12-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-43-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-43-1.png
/med

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-41-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-60-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-60-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-60-2.png
/med

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-67-11.png


In [13]:
from math import sin, cos, sqrt, atan2, radians

def getGreatCircleDistance(lat1,lon1,lat2,lon2):
    R = 6371.009
    
    lat1 = radians(abs(lat1))
    lon1 = radians(abs(lon1))
    lat2 = radians(abs(lat2))
    lon2 = radians(abs(lon2))
    
    delta_lon = lon2 - lon1
    delta_lat = lat2 - lat1

    a = sin(delta_lat / 2)**2 + cos(lat1) * cos(lat2) * sin(delta_lon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
print(getGreatCircleDistance(41.53283, -8.81532, 41.53101, -8.80974))

In [15]:
labeled_data = pd.read_csv('/home/diegues/Desktop/ProcessedImages/labeled_data.csv')
labeled_data.head()
        

,filename,timestamp,latitude,longitude,roll,pitch,entropy,date,depth,EunisCode,EunisName,level1,level2,level3,level4,level5,level6,species,AphiaID
0,104728_cam-np3_frame5.jpg,1.525172e+09,41.53283,-8.81532,-3.36,-14.12,21.934646,01/05/18,16.97,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Mesophyllum lichenoides,145188.0
1,104728_cam-np3_frame9.jpg,1.525172e+09,41.53283,-8.81531,-2.34,-16.76,21.932552,01/05/18,16.95,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Mesophyllum lichenoides,145188.0
2,104728_cam-np3_frame50.jpg,1.525172e+09,41.53277,-8.81526,-25.75,-17.73,21.933617,01/05/18,16.11,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Mesophyllum lichenoides,145188.0
3,104728_cam-np3_frame51.jpg,1.525172e+09,41.53277,-8.81526,-25.75,-17.73,21.934235,01/05/18,16.11,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Mesophyllum lichenoides,145188.0
4,104728_cam-np3_frame52.jpg,1.525172e+09,41.53277,-8.81526,-25.75,-17.73,21.909359,01/05/18,16.11,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Echinus esculentus,124287.0


In [18]:
level3 = {}
imagedata = []
for index, row in labeled_data.iterrows():
    for index2, row2 in df.iterrows():
        distance = getGreatCircleDistance(row['latitude'], row['longitude'], row2['latitude'], row2['longitude'])
        #print(row['filename'],row2['filename'],distance)
        if distance <= 0.01:
            level3[row2['filename']] = row['level3']
            imagedata.append(row['filename'] + ',' + row2['filename'] + ',' + row['level3'] + ',' + str(distance))

KeyboardInterrupt: 

In [ ]:
len(level3)


In [ ]:
len(imagedata)

In [ ]:
lcl_df = pd.DataFrame(columns=['photo_name', 'sss_name', 'level3', 'distance'])

In [ ]:
index = 0
for row in imagedata:
    lcl_df.loc[index] = row.split(',')
    index += 1
    

In [ ]:
lcl_df.head()

In [41]:
#lcl_df.to_csv(os.path.join(new_folder,'local_match.csv'),index=False)
lcl_df = pd.read_csv(os.path.join(new_folder,'local_match.csv'))

In [42]:
lcl_df.head()

,photo_name,sss_name,level3,distance
0,104728_cam-np3_frame5.jpg,105317_cam-np3-1-9,A4.1,0.007249
1,104728_cam-np3_frame5.jpg,105317_cam-np3-1-10,A4.1,0.006709
2,104728_cam-np3_frame5.jpg,105317_cam-np3-2-11,A4.1,0.001584
3,104728_cam-np3_frame5.jpg,105317_cam-np3-2-2,A4.1,0.004862
4,104728_cam-np3_frame5.jpg,112045_mb_survey-1-0,A4.1,0.009011


In [43]:
# escolher a class mais proxima do ponto central 
new_levels = pd.get_dummies(lcl_df.level3)
new_levels.head()

,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0


In [44]:
#lcl_df = lcl_df.drop('level3', axis=1)
lcl_df = pd.concat([lcl_df, new_levels], axis=1)
lcl_df.head()

,photo_name,sss_name,level3,distance,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
0,104728_cam-np3_frame5.jpg,105317_cam-np3-1-9,A4.1,0.007249,0,0,1,0,0,0,0
1,104728_cam-np3_frame5.jpg,105317_cam-np3-1-10,A4.1,0.006709,0,0,1,0,0,0,0
2,104728_cam-np3_frame5.jpg,105317_cam-np3-2-11,A4.1,0.001584,0,0,1,0,0,0,0
3,104728_cam-np3_frame5.jpg,105317_cam-np3-2-2,A4.1,0.004862,0,0,1,0,0,0,0
4,104728_cam-np3_frame5.jpg,112045_mb_survey-1-0,A4.1,0.009011,0,0,1,0,0,0,0


In [45]:
min_dist = lcl_df[['sss_name', 'level3', 'distance']].groupby('sss_name').min()
min_dist.head()

,level3,distance
sss_name,,
104728_cam-np3-1-4,A4.7,0.007185
104728_cam-np3-1-5,A4.1,0.007478
104728_cam-np3-2-10,A4.1,0.007234
104728_cam-np3-2-8,A4.7,0.008970
104728_cam-np3-2-9,A4.1,0.007015


In [46]:
sss_classes = lcl_df.drop(['photo_name', 'level3', 'distance'], axis = 1).groupby('sss_name').sum()
sss_classes.head()

,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
sss_name,,,,,,,
104728_cam-np3-1-4,0,0,0,57,0,0,0
104728_cam-np3-1-5,0,0,3,29,0,0,0
104728_cam-np3-2-10,0,0,26,0,0,0,0
104728_cam-np3-2-8,0,0,0,57,0,0,0
104728_cam-np3-2-9,0,0,26,52,0,0,0


In [47]:
sss_df = pd.concat([min_dist, sss_classes], axis = 1)
sss_df

,level3,distance,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
sss_name,,,,,,,,,
104728_cam-np3-1-4,A4.7,0.007185,0,0,0,57,0,0,0
104728_cam-np3-1-5,A4.1,0.007478,0,0,3,29,0,0,0
104728_cam-np3-2-10,A4.1,0.007234,0,0,26,0,0,0,0
104728_cam-np3-2-8,A4.7,0.008970,0,0,0,57,0,0,0
104728_cam-np3-2-9,A4.1,0.007015,0,0,26,52,0,0,0
105317_cam-np3-1-10,A4.1,0.005886,0,0,28,52,0,0,0
105317_cam-np3-1-11,A4.7,0.005251,0,0,0,57,0,0,0
105317_cam-np3-1-2,A4.1,0.005842,0,0,3,29,0,0,0
105317_cam-np3-1-3,A4.7,0.004719,0,0,0,57,0,0,0


In [53]:
sss_df.columns = ['min_distance_class', 'min_distance'] + list(sss_df.columns[2:])
sss_df.head()

,min_distance_class,min_distance,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
sss_name,,,,,,,,,
104728_cam-np3-1-4,A4.7,0.007185,0,0,0,57,0,0,0
104728_cam-np3-1-5,A4.1,0.007478,0,0,3,29,0,0,0
104728_cam-np3-2-10,A4.1,0.007234,0,0,26,0,0,0,0
104728_cam-np3-2-8,A4.7,0.008970,0,0,0,57,0,0,0
104728_cam-np3-2-9,A4.1,0.007015,0,0,26,52,0,0,0


In [54]:
sss_df.to_csv(os.path.join(new_folder,'sss_class.csv'))

In [55]:
len(sss_df)

420